# 2022-09-15 • Two-pass connection test: ptp, then corr with found avg

Two-pass connection test: peak-to-peak, then correlation with found average

## Imports

In [1]:
#

In [2]:
using MyToolbox

In [3]:
using VoltoMapSim

[ Info: Precompiling VoltoMapSim [f713100b-c48c-421a-b480-5fcb4c589a9e]
[ Info: Skipping precompilation since __precompile__(false). Importing VoltoMapSim [f713100b-c48c-421a-b480-5fcb4c589a9e].
[ Info: Precompiling Sciplotlib [61be95e5-9550-4d5f-a203-92a5acbc3116]


## Params

In [4]:
p = get_params(
    duration = 10minutes,
    p_conn = 0.04,
    g_EE = 1,
    g_EI = 1,
    g_IE = 4,
    g_II = 4,
    ext_current = Normal(-0.5 * pA/√seconds, 5 * pA/√seconds),
    E_inh = -80 * mV,
    record_v = [1:40; 801:810],
);

## Run sim

In [5]:
@time s = cached(sim, [p.sim]);

  9.931489 seconds (14.15 M allocations: 3.174 GiB, 11.45% gc time, 67.93% compilation time)


(To speed this up (precompile this further): investigate with SnoopCompile or JET. Not now).

In [6]:
s = augment(s, p);

## First pass: peak-to-peak

Let's start with just neuron 1 as postsyn.

In [9]:
m = 1;

In [10]:
perf = cached_conntest_eval(s,m,p);

In [12]:
p.conntest.STA_test_statistic

"ptp"

In [21]:
ENV["LINES"] = 4
perf.tested_neurons

,input_neuron_ID,real_type,predicted_type,pval,area_over_start
,Int64,Symbol,Symbol,Float64,Float64
1,139,exc,exc,0.01,17
2,681,exc,inh,0.01,-4
3,11,exc,exc,0.01,18
4,766,exc,exc,0.01,52
⋮,⋮,⋮,⋮,⋮,⋮


We take all inputs with `predicted_type` :exc.

Current pval threshold is 0.05.  
We could be stricter.

It'll be tradeoff: stricter threshold gives less STAs to average to build our template;
but there will be less noisy STAs mixed in (or even wrong STAs, i.e. of non-inputs).

In [17]:
#= implementation note: 
it's a two stage process, so maybe split dataframe.
namely:

(inputID, postsynID) -> (pval, area_over_start) -> predicted_type

first step depends on 'test statistics' used (ptp, corr-with-some-template).
second step depends on α.

(between first and second data, there's a "-> STA ->" step.
But we don't save that STA in dataframe, too large. it's implicit).

The large work btw is the first step: the shuffles, and for each calculating STA.
So.. we could indeed store these shuffled STAs.. it's just 1000 samples per.
=#